# Method2: Run OWL using MF slice with a meas_node

## Prerequisites

Create an MF slice with 3 nodes using `KNIT6_prepare_a_slice_via_images.ipynb` and name it **"MFLibKNIT6"**

## Imports
This series of notebooks all need a common set of imports which are defined in [Common Imports](./KNIT6_common_imports.ipynb)

**slice_name** is defined in this step. If you would like to change the slice_name, edit [Common Imports](./KNIT6_common_imports.ipynb)

In [ ]:
%run "./KNIT6_common_imports.ipynb"

In [ ]:
from mflib import owl

In [ ]:
from configparser import ConfigParser
from pathlib import Path

## Identify a slice

In [ ]:
#slice_name="MFLibKNIT6"

try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(node) 

except Exception as e:
    print(f"Exception: {e}")

### Get Nodes

In [ ]:
node1 = slice.get_node("Node1")
node2 = slice.get_node("Node2")
node3 = slice.get_node("Node3")
nodes = slice.get_nodes()

## If MF has not been initialized, run the following cell

Among other things, it downloads OWL service scripts to meas_node. As you will see below, those scripts are used to orchestrate OWL measurement sessions on Experimenters' nodes.


In [ ]:
mf=MFLib(slice_name)

## Set up OWL

### Create necessary local directories

(If there are already directories with the same name, it will print warning)

In [ ]:
local_owl_dir=f'/home/fabric/work/owl_output/{slice_name}'

In [ ]:
owl_mf = owl.OwlMf(local_owl_dir = local_owl_dir)

# Prepare local OWL service files

### (optional) Look up experiment IP address(es) and site for each node

In [ ]:
for node in nodes:
    print(f"{node.get_name()}: {owl_mf.list_experiment_ip_addrs(node)} at {node.get_site()}")

In [ ]:
node1_ip = owl_mf.list_experiment_ip_addrs(node1)[0]
node2_ip = owl_mf.list_experiment_ip_addrs(node2)[0]
node3_ip = owl_mf.list_experiment_ip_addrs(node3)[0]

print(node1_ip, node2_ip, node3_ip)

### Write config and links files locally

#### Current options for `owl.conf` :

- `send_int (default=0.5)`: interval (sec) at which UDP probe packets are sent. Values smaller than 0.5 may strain the system (we are working on this)
- `port (default=5005)`: Leave it as it is unless there is a conflict
- `cap_mode (default="save")`: Currently that is the only option
- `pcap_int (default=120)`: interval at which a new pcap file will be started (to limit the size of each pcap file)


In [ ]:
owl_mf.generate_local_config(send_int=1.0, pcap_int=280)

#### Identify (src_ip, dst_ip) pairs for running OWL and write to `links.json` file

In [ ]:
owl_mf.generate_local_links_file([(node2_ip, node1_ip), 
                                  (node1_ip, node2_ip), 
                                  (node3_ip, node2_ip), 
                                  (node1_ip, node3_ip)])

### (Optional) Print local service files to confirm the selection

In [ ]:
owl_mf.print_local_service_files()

## (Optional) Check output files from previous runs on all remote nodes

In [ ]:
for node in nodes:
    print(f"\n{node.get_name()}:")
    owl_mf.list_remote_output(node)

### Delete if necessary

In [ ]:
for node in nodes:
    print(f"\n{node.get_name()}:")
    owl_mf.delete_remote_output(node)

# Set up OWL on remote nodes via meas_node

### MF: Create OWL on the slice 

#### This command will tell meas_node to ... 
- Create output and config directories on Exp nodes
- Pull OWL image from Docker Hub

In [ ]:
mf.create("owl")

### MF: Upload the config and links files to the meas_node 

#### Find local service file paths

In [ ]:
owl_files = owl_mf.get_local_service_file_paths()
print(owl_files)

#### Copy the files to meas_node

In [ ]:
mf.update(service="owl", files=owl_files)

#### (optional) Check the meas_node copy looks right

In [ ]:
mf.info("owl")

# MF: Start OWL

#### Meas_node will 
- Copy the updated owl.conf and links.json files to all experimenter nodes
- Start docker containers on all experimenter nodes

In [ ]:
mf.start(services=["owl",])

# MF: Stop OWL

#### Meas_node will 
- Stop runnning OWL docker containers on all experimenter nodes

In [ ]:
mf.stop(services=["owl",])

# Download output

- Downloads the generated `*.pcap` files from all nodes to  `<specified owl_dir>/output/<node_name>`

### (optional) Check if pcap files have been generated

In [ ]:
for node in nodes:
    print(f"\n{node.get_name()}")
    owl_mf.list_remote_output(node)

### Download

In [ ]:
for node in nodes:
    print(f"\n{node.get_name()}")
    owl_mf.download_output(node)

## ⚠️☠️ Run ONLY IF you want to remove OWL

In [ ]:
#mf.remove(services=["owl",])